In [2]:
"""
SPECTRUM: Ferramenta de Análise e Correção de Imagens
Projeto de Processamento de Imagens - Cidade de Véridia

Grupo: Erika Dias do Nascimento Marques, Cauan Lemos Barbosa Paulo,
       Paulo Souza Telles Filho, Thiago Vinny, Leandro Dias

Funcionalidades:
- Transformações Lineares (Brilho/Contraste) - Leandro Dias
- Transformações Não-Lineares (Gamma, Log) - Thiago Vinny
- Equalização de Histograma - Erika Dias
- Análise de Histograma - Cauan Lemos
- Visualização Comparativa - Paulo Telles
- Parâmetros Personalizados - Erika Dias
"""

import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt
from skimage import exposure, io, color
import os
from datetime import datetime
import json
from typing import Dict, List, Tuple, Any
from google.colab import files, drive
import ipywidgets as widgets
from IPython.display import display, clear_output
import warnings
warnings.filterwarnings('ignore')

class SpectrumImageProcessor:
    """
    Sistema principal de processamento de imagens Spectrum
    Centraliza todas as funcionalidades de análise e correção
    """

    def __init__(self):
        self.images_history = []
        self.current_image = None
        self.original_image = None
        self.processed_image = None
        self.user_type = "operador"

        print("SPECTRUM - Sistema de Processamento de Imagens")
        print("=" * 50)

    def upload_image(self):
        """
        Funcional 1: Permite upload de imagens para processamento
        Suporta formatos: JPEG, PNG, TIFF, BMP
        """
        print("📤 Carregando imagem...")
        uploaded = files.upload()

        if uploaded:
            filename = list(uploaded.keys())[0]

            # Verificar formato suportado (Funcional 12)
            valid_formats = ['.jpg', '.jpeg', '.png', '.tiff', '.bmp']
            if not any(filename.lower().endswith(fmt) for fmt in valid_formats):
                print("Formato não suportado!")
                return False

            # Carregar imagem
            self.original_image = cv2.imread(filename)
            if self.original_image is None:
                print("Erro ao carregar a imagem!")
                return False

            self.current_image = self.original_image.copy()

            # Registrar log (Funcional 15)
            self._log_operation("upload", {"filename": filename, "shape": self.original_image.shape})

            print(f"Imagem carregada: {filename}")
            print(f"Dimensões: {self.original_image.shape}")
            return True

        return False

    def linear_transformations(self, brightness=0, contrast=1.0):
        """
        Funcional 2: Transformações Lineares - Leandro Dias
        Ajuste de brilho e contraste com modificação direta dos pixels

        Parâmetros:
        - brightness: valor a ser adicionado (-100 a 100)
        - contrast: fator multiplicativo (0.1 a 3.0)
        """
        if self.current_image is None:
            print("Carregue uma imagem primeiro!")
            return

        print(f"Aplicando transformação linear (Brilho: {brightness}, Contraste: {contrast:.2f})")

        # Aplicar transformação linear: new_pixel = contrast * pixel + brightness
        self.processed_image = cv2.convertScaleAbs(self.current_image, alpha=contrast, beta=brightness)

        # Registrar transformação (Funcional 8)
        self._log_operation("linear_transform", {
            "brightness": brightness,
            "contrast": contrast,
            "timestamp": datetime.now().isoformat()
        })

        print("Transformação linear aplicada")
        self.current_image = self.processed_image.copy()

    def nonlinear_transformations(self, transformation_type="gamma", gamma=1.0, c_log=1.0):
        """
        Funcional 3: Transformações Não-Lineares - Thiago Vinny
        Inclui correção gamma e funções logarítmicas

        Parâmetros:
        - transformation_type: 'gamma', 'log', 'power'
        - gamma: valor para correção gamma (0.1 a 3.0)
        - c_log: constante para transformação logarítmica
        """
        if self.current_image is None:
            print("Carregue uma imagem primeiro!")
            return

        print(f"Aplicando transformação não-linear: {transformation_type}")

        # Normalizar imagem para 0-1
        img_normalized = self.current_image.astype(np.float64) / 255.0

        if transformation_type == "gamma":
            # Correção Gamma: s = c * r^γ
            self.processed_image = np.power(img_normalized, gamma)

        elif transformation_type == "log":
            # Transformação Logarítmica: s = c * log(1 + r)
            self.processed_image = c_log * np.log1p(img_normalized)

        elif transformation_type == "power":
            # Transformação de Potência
            self.processed_image = np.power(img_normalized, 1/gamma)

        # Converter de volta para 0-255
        self.processed_image = np.clip(self.processed_image * 255, 0, 255).astype(np.uint8)

        # Registrar transformação
        self._log_operation("nonlinear_transform", {
            "type": transformation_type,
            "gamma": gamma,
            "c_log": c_log,
            "timestamp": datetime.now().isoformat()
        })

        print("Transformação não-linear aplicada!")
        self.current_image = self.processed_image.copy()

    def histogram_equalization(self, method="global"):
        """
        Funcional 4: Equalização de Histograma - Erika Dias
        Melhora distribuição de tons para imagens com baixo contraste

        Parâmetros:
        - method: 'global' ou 'adaptive' (CLAHE)
        """
        if self.current_image is None:
            print("Carregue uma imagem primeiro!")
            return

        print(f"Aplicando equalização de histograma: {method}")

        if len(self.current_image.shape) == 3:
            # Imagem colorida - converter para YUV e equalizar apenas Y
            yuv = cv2.cvtColor(self.current_image, cv2.COLOR_BGR2YUV)

            if method == "global":
                yuv[:,:,0] = cv2.equalizeHist(yuv[:,:,0])
            else:  # adaptive
                clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
                yuv[:,:,0] = clahe.apply(yuv[:,:,0])

            self.processed_image = cv2.cvtColor(yuv, cv2.COLOR_YUV2BGR)
        else:
            # Imagem em escala de cinza
            if method == "global":
                self.processed_image = cv2.equalizeHist(self.current_image)
            else:  # adaptive
                clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
                self.processed_image = clahe.apply(self.current_image)

        # Registrar transformação
        self._log_operation("histogram_equalization", {
            "method": method,
            "timestamp": datetime.now().isoformat()
        })

        print("Equalização de histograma aplicada!")
        self.current_image = self.processed_image.copy()

    def analyze_histogram(self, show_comparison=True):
        """
        Funcional 5: Análise de Histograma - Cauan Lemos
        Exibe histogramas antes e depois das transformações
        """
        if self.original_image is None:
            print("Carregue uma imagem primeiro!")
            return

        print("Analisando histogramas...")

        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        fig.suptitle('SPECTRUM - Análise de Histogramas', fontsize=16, fontweight='bold')

        # Imagem original
        if len(self.original_image.shape) == 3:
            axes[0,0].imshow(cv2.cvtColor(self.original_image, cv2.COLOR_BGR2RGB))
        else:
            axes[0,0].imshow(self.original_image, cmap='gray')
        axes[0,0].set_title('Imagem Original')
        axes[0,0].axis('off')

        # Histograma original
        self._plot_histogram(self.original_image, axes[0,1], "Histograma Original")

        if show_comparison and self.processed_image is not None:
            # Imagem processada
            if len(self.processed_image.shape) == 3:
                axes[1,0].imshow(cv2.cvtColor(self.processed_image, cv2.COLOR_BGR2RGB))
            else:
                axes[1,0].imshow(self.processed_image, cmap='gray')
            axes[1,0].set_title('Imagem Processada')
            axes[1,0].axis('off')

            # Histograma processado
            self._plot_histogram(self.processed_image, axes[1,1], "Histograma Processado")

        plt.tight_layout()
        plt.show()

        # Gerar estatísticas (Funcional 10)
        self._generate_statistics()

    def comparative_visualization(self):
        """
        Funcional 6: Visualização Comparativa - Paulo Telles
        Interface para comparar imagem original e transformada lado a lado
        """
        if self.original_image is None or self.processed_image is None:
            print("Execute uma transformação primeiro!")
            return

        print("Visualização Comparativa")

        fig, axes = plt.subplots(1, 2, figsize=(16, 6))
        fig.suptitle('SPECTRUM - Comparação Lado a Lado', fontsize=16, fontweight='bold')

        # Imagem original
        if len(self.original_image.shape) == 3:
            axes[0].imshow(cv2.cvtColor(self.original_image, cv2.COLOR_BGR2RGB))
        else:
            axes[0].imshow(self.original_image, cmap='gray')
        axes[0].set_title('Original')
        axes[0].axis('off')

        # Imagem processada
        if len(self.processed_image.shape) == 3:
            axes[1].imshow(cv2.cvtColor(self.processed_image, cv2.COLOR_BGR2RGB))
        else:
            axes[1].imshow(self.processed_image, cmap='gray')
        axes[1].set_title('Processada')
        axes[1].axis('off')

        plt.tight_layout()
        plt.show()

    def custom_parameters_interface(self):
        """
        Funcional 7: Ajustes Manuais de Parâmetros - Erika Dias
        Interface interativa para ajuste de brilho, contraste e gamma
        """
        print("Interface de Parâmetros Personalizados")

        # Sliders interativos
        brightness_slider = widgets.IntSlider(
            value=0, min=-100, max=100, step=1,
            description='Brilho:'
        )

        contrast_slider = widgets.FloatSlider(
            value=1.0, min=0.1, max=3.0, step=0.1,
            description='Contraste:'
        )

        gamma_slider = widgets.FloatSlider(
            value=1.0, min=0.1, max=3.0, step=0.1,
            description='Gamma:'
        )

        def update_image(brightness, contrast, gamma):
            if self.original_image is not None:
                # Aplicar transformações
                temp_image = cv2.convertScaleAbs(self.original_image, alpha=contrast, beta=brightness)

                # Aplicar gamma
                temp_normalized = temp_image.astype(np.float64) / 255.0
                temp_gamma = np.power(temp_normalized, gamma)
                self.processed_image = np.clip(temp_gamma * 255, 0, 255).astype(np.uint8)
                self.current_image = self.processed_image.copy()

                # Mostrar resultado
                self.comparative_visualization()

        # Widget interativo
        interactive_widget = widgets.interactive(
            update_image,
            brightness=brightness_slider,
            contrast=contrast_slider,
            gamma=gamma_slider
        )

        display(interactive_widget)

    def generate_report(self):
        """
        Funcional 9: Gerar relatórios detalhados sobre transformações
        """
        if not self.images_history:
            print("Nenhuma operação registrada!")
            return

        print("Gerando Relatório Detalhado...")

        report = {
            "sistema": "SPECTRUM - Ferramenta de Análise e Correção de Imagens",
            "cidade": "Véridia",
            "data_relatorio": datetime.now().isoformat(),
            "total_operacoes": len(self.images_history),
            "operacoes": self.images_history,
            "estatisticas": self._get_processing_stats()
        }

        # Salvar relatório
        filename = f"spectrum_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(report, f, indent=2, ensure_ascii=False)

        print(f"✅ Relatório salvo: {filename}")

        # Exibir sumário
        print("\n" + "="*50)
        print("RESUMO DO RELATÓRIO")
        print("="*50)
        print(f"Total de operações: {len(self.images_history)}")

        # Contar tipos de operações
        op_types = {}
        for op in self.images_history:
            op_type = op['operation']
            op_types[op_type] = op_types.get(op_type, 0) + 1

        for op_type, count in op_types.items():
            print(f"- {op_type}: {count} vezes")

        return filename

    def download_processed_image(self, filename=None):
        """
        Funcional 13: Download das imagens processadas
        """
        if self.processed_image is None:
            print("Nenhuma imagem processada disponível!")
            return

        if filename is None:
            filename = f"spectrum_processed_{datetime.now().strftime('%Y%m%d_%H%M%S')}.png"

        # Salvar imagem
        if len(self.processed_image.shape) == 3:
            cv2.imwrite(filename, self.processed_image)
        else:
            cv2.imwrite(filename, self.processed_image)

        print(f"Imagem salva: {filename}")

        # Download no Colab
        files.download(filename)

    def workflow_demo(self):
        """
        Demonstração completa do fluxo de processamento
        Atende aos requisitos específicos do projeto
        """
        print("DEMONSTRAÇÃO DO SISTEMA SPECTRUM")
        print("=" * 50)

        # 1. Upload de imagem
        print("\n1️⃣ UPLOAD DE IMAGEM")
        if not self.upload_image():
            print("Demo interrompida - erro no upload")
            return

        # 2. Análise inicial
        print("\n2️⃣ ANÁLISE INICIAL")
        self.analyze_histogram(show_comparison=False)

        # 3. Transformações lineares
        print("\n3️⃣ TRANSFORMAÇÕES LINEARES")
        self.linear_transformations(brightness=20, contrast=1.2)

        # 4. Transformações não-lineares
        print("\n4️⃣ TRANSFORMAÇÕES NÃO-LINEARES")
        self.nonlinear_transformations("gamma", gamma=0.8)

        # 5. Equalização de histograma
        print("\n5️⃣ EQUALIZAÇÃO DE HISTOGRAMA")
        self.histogram_equalization("adaptive")

        # 6. Análise comparativa
        print("\n6️⃣ ANÁLISE COMPARATIVA")
        self.analyze_histogram()
        self.comparative_visualization()

        # 7. Relatório
        print("\n7️⃣ GERAÇÃO DE RELATÓRIO")
        report_file = self.generate_report()

        print("\nDEMONSTRAÇÃO COMPLETA!")
        print(f"Relatório: {report_file}")

    # Métodos auxiliares
    def _plot_histogram(self, image, ax, title):
        """Plota histograma de uma imagem"""
        if len(image.shape) == 3:
            colors = ['blue', 'green', 'red']
            for i, color in enumerate(colors):
                hist = cv2.calcHist([image], [i], None, [256], [0, 256])
                ax.plot(hist, color=color, alpha=0.7)
        else:
            hist = cv2.calcHist([image], [0], None, [256], [0, 256])
            ax.plot(hist, color='black')

        ax.set_title(title)
        ax.set_xlabel('Intensidade')
        ax.set_ylabel('Pixels')
        ax.grid(True, alpha=0.3)

    def _log_operation(self, operation, parameters):
        """Registra operação no histórico (Funcional 15)"""
        log_entry = {
            "operation": operation,
            "parameters": parameters,
            "timestamp": datetime.now().isoformat(),
            "user_type": self.user_type
        }
        self.images_history.append(log_entry)

    def _generate_statistics(self):
        """Gera estatísticas das imagens"""
        if self.original_image is not None:
            print("\nESTATÍSTICAS DA IMAGEM")
            print("-" * 30)

            # Original
            orig_mean = np.mean(self.original_image)
            orig_std = np.std(self.original_image)
            print(f"Original - Média: {orig_mean:.2f}, Desvio: {orig_std:.2f}")

            # Processada (se existir)
            if self.processed_image is not None:
                proc_mean = np.mean(self.processed_image)
                proc_std = np.std(self.processed_image)
                print(f"Processada - Média: {proc_mean:.2f}, Desvio: {proc_std:.2f}")
                print(f"Diferença - Média: {proc_mean - orig_mean:.2f}")

    def _get_processing_stats(self):
        """Retorna estatísticas de processamento"""
        if not self.images_history:
            return {}

        stats = {
            "primeira_operacao": self.images_history[0]["timestamp"],
            "ultima_operacao": self.images_history[-1]["timestamp"],
            "tipos_operacao": {}
        }

        for op in self.images_history:
            op_type = op["operation"]
            if op_type not in stats["tipos_operacao"]:
                stats["tipos_operacao"][op_type] = 0
            stats["tipos_operacao"][op_type] += 1

        return stats

def main():
    """
    Função principal para demonstrar o sistema Spectrum
    """
    print("=" * 50)

    # Criar instância do processador
    spectrum = SpectrumImageProcessor()

    # Menu de opções
    while True:
        print("\nMENU - Escolha uma opção:")
        print("1 - Upload de Imagem")
        print("2 - Transformações Lineares")
        print("3 - Transformações Não-Lineares")
        print("4 - Equalização de Histograma")
        print("5 - Análise de Histograma")
        print("6 - Visualização Comparativa")
        print("7 - Parâmetros Personalizados")
        print("8 - Demonstração Completa")
        print("9 - Gerar Relatório")
        print("10 - Download Imagem")
        print("0 - Sair")

        choice = input("\nEscolha uma opção: ")

        if choice == "1":
            spectrum.upload_image()
        elif choice == "2":
            brilho = int(input("Brilho (-100 a 100): ") or "0")
            contraste = float(input("Contraste (0.1 a 3.0): ") or "1.0")
            spectrum.linear_transformations(brilho, contraste)
        elif choice == "3":
            tipo = input("Tipo (gamma/log/power): ") or "gamma"
            gamma = float(input("Valor gamma (0.1 a 3.0): ") or "1.0")
            spectrum.nonlinear_transformations(tipo, gamma)
        elif choice == "4":
            metodo = input("Método (global/adaptive): ") or "global"
            spectrum.histogram_equalization(metodo)
        elif choice == "5":
            spectrum.analyze_histogram()
        elif choice == "6":
            spectrum.comparative_visualization()
        elif choice == "7":
            spectrum.custom_parameters_interface()
        elif choice == "8":
            spectrum.workflow_demo()
        elif choice == "9":
            spectrum.generate_report()
        elif choice == "10":
            spectrum.download_processed_image()
        elif choice == "0":
            print("Processo encerrado!")
            break
        else:
            print("Opção inválida!")

if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'cv2'